In [1]:
import camelot

In [2]:
import glob

In [3]:
directory = '/Users/joqsan/0-Excel/Reading_From_Folders/FONASA/*.pdf'

In [4]:
directory

'/Users/joqsan/0-Excel/Reading_From_Folders/FONASA/*.pdf'

In [5]:
files = [filename for filename in glob.glob(directory)]

In [6]:
files

['/Users/joqsan/0-Excel/Reading_From_Folders/FONASA/Fonasa_01.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/FONASA/Fonasa_02.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/FONASA/Fonasa_03.pdf']

## Let's test FONASA

In [7]:
import camelot
import pandas as pd
%matplotlib notebook
import re
import calendar
import locale
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

'es_ES.UTF-8'

In [8]:
months = list(calendar.month_abbr)
months = list(map(lambda x: x.capitalize(), months))

In [9]:
area = ['54, 288, 294, 158', '60, 335, 131, 301']

In [10]:
def transpose_data(df, month, name):
    df = df.transpose()
    df.columns = df.iloc[0, :]
    df = df.drop(0, axis=0)
    df = df.set_index([[month], [name]])
    
    return df

In [11]:
def fonasa_data_cleaning(dfs):
    
    part1 = dfs[0].df.copy()
    month_str = part1.iloc[1, 0]
    month_id = int(re.sub(r'0([0-9])', r'\1', month_str))
    month = months[month_id]
    
    
    part2 = dfs[1].df.copy()
    part2.drop(0, inplace=True)
    part2.drop(1, axis=1, inplace=True)
    part2.iloc[-1, 0] = 'Total A Pagar'
    
    
    part2.iloc[:, [1]] = part2.iloc[:, [1]].apply(lambda x: x.str.replace('.', '')).astype(int)
    
    fonasa = transpose_data(part2, month, 'FONASA')
    
    return fonasa

In [13]:
fonasa_list = []
for fonasa_file in files:
    fonasa_dfs = camelot.read_pdf(fonasa_file, pages='1', flavor='stream', table_areas=area)
    fonasa = fonasa_data_cleaning(fonasa_dfs)
    fonasa_list.append(fonasa)

In [15]:
df = pd.concat(fonasa_list)

In [16]:
df

,,Cotiz. Legal,Reajuste,Interés,Multas,SUB TOTAL FONASA,Compensación Asignación Familiar,Total A Pagar
Ene,FONASA,905198,0,0,0,905198,0,905198
Feb,FONASA,955785,0,0,0,955785,0,955785
Mar,FONASA,1089890,0,0,0,1089890,0,1089890


In [17]:
df.sum(axis=0)

0
Cotiz. Legal                        2950873.0
Reajuste                                  0.0
Interés                                   0.0
Multas                                    0.0
SUB TOTAL FONASA                    2950873.0
Compensación Asignación Familiar          0.0
Total A Pagar                       2950873.0
dtype: float64